In [ ]:
import matplotlib.pyplot as plt
from numpy import arange, exp, pi, zeros, correlate, array
from scipy.signal import find_peaks
%matplotlib widget

In [ ]:
def autocorrelation(type, data):
    """Doing correlation to the signal itself using designed mathematics equation. Time shifted to the length of data.

    Parameters
    ----------
    type : String
        Function type: normal for using the direct correlation, inversefourier using magnitude spectrum
    data : 1D array
        Input of the function, data signal

    Returns
    -------
    1D array
        Return of the autocorrelation
    """
    result = array(zeros(len(data)), dtype=complex)
    magnitudespectrum = array(zeros(len(data)), dtype=complex)
    if type == "normal":
        return [x/len(data) for x in correlate(data, data, mode="same")]
    elif type == "inversefourier":
        for i in range(len(data)):
            for j in range(len(data)):
                magnitudespectrum[i] += (data[j]**2)*exp(-2j*pi*i*j/len(data))
        # magnitudespectrum = data
        
        for i in range(len(data)):
            for j in range(len(data)):
                result[i] += (magnitudespectrum[j]**2)*exp(2j*pi*i*j/len(magnitudespectrum))
            result[i] = result[i]/len(magnitudespectrum)
        return result
    else: raise Exception("Please check the available type!") 

In [ ]:
def crosscorrelation(firstdata, seconddata):
    """Doing correlation on two signal using designed mathematical equation. Time shifted to the first signal. 

    Parameters
    ----------
    firstdata : 1D Array
        Base signal for correlation. The signal that doesn't move on the line.
    seconddata : 1D Array
        Signal for the correlation, signal move across the first signal.

    Returns
    -------
    1D Array
        Return of the function with length of the first signal.
    """
    result=zeros(len(firstdata))

    for i in range(len(firstdata)):
        for j in range(len(seconddata)):
            if i + j >= len(firstdata):
                break
            result[i]=result[i]+abs(result[i]+firstdata[i]*seconddata[j]*exp(2j*pi*i*j/len(firstdata)))
        result[i]=abs(result[i])/len(firstdata)

    return result

In [ ]:
def ecgpeakfinding(type, data):
    """Finding the peak position of the electrocardiogram signal based on the correlation mathematical function method.

    Parameters
    ----------
    type : String
        Function type: normal for using the direct correlation, inversefourier using magnitude spectrum
    data : 1D Array
        Input array of electrocardiogram signal

    Returns
    -------
    Tuple [1D Array, 1D Array]
        Return the peak position and the peak value of the electrocardiogram signal.
    """
    correlation = autocorrelation(type, data)
    peakposition = find_peaks(correlation, distance=100, height=0.1)[0]

    peakvalue = [correlation[i] for i in peakposition]
    return peakposition, peakvalue

In [ ]:
def onchangeselectfirstsignal():
    pass

def onchangeselectsecondsignal():
    pass

In [ ]:
texts0 = [[], []] # X(sequence) and Y(ecg signal) value

with open("/home/muhammadakbarmaulana/project-release/auto-and-cross-correlation/ECG Non-Filtered 1000 Data.txt", "r") as file:
  textfile = file.readlines()


texts0[0] = arange(len(textfile))

for text in textfile: 
  texts0[1].append(float(text.replace("\n", "")))

fig = plt.figure()
ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="ElectroCardioGram Signal")
ax0.plot(texts0[0], texts0[1])
plt.show()

In [ ]:
texts1 = [[], []] # X(sequence) and Y(ecg signal) value

with open("/home/muhammadakbarmaulana/project-release/auto-and-cross-correlation/ECG Non-Filtered 1000 Data.txt", "r") as file:
  textfile = file.readlines()


texts1[0] = arange(len(textfile))

for text in textfile: 
  texts1[1].append(float(text.replace("\n", "")))

fig = plt.figure()
ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="ElectroCardioGram Signal")
ax0.plot(texts1[0], texts1[1])
plt.show()

In [ ]:
fig = plt.figure()
ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Auto Correlation")
ax0.plot(texts0[0], texts0[1], label="ECG Signal")
ax0.legend()
ax1 = ax0.twinx()
ax1.set_ylabel(ylabel="amplitude")
ax1.plot(texts0[0], [abs(x) for x in autocorrelation("inversefourier", texts0[1])], label="Inverse Fourier", color="green")
ax1.legend()
plt.show()

In [ ]:
fig = plt.figure()
result = correlate(texts0[1], [x/len(texts0[1]) for x in texts0[1]], mode="full")[:1000]
ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude")
ax0.plot(arange(len(result)), result, label="Correlated Signal")
ax0.legend()
plt.show()

In [ ]:
from pathlib import Path
from os import listdir
print("File      Path: {}".format(Path().resolve()))
print("Directory Path: {}".format(Path().absolute()))
files = listdir(Path().absolute())
for f in files:
	print(f)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

print("Please select the same data length for first and second signal")

selectfirstsignal = widgets.Select(
    options=files,
    description='Select First File:',
    disabled=False
)

firstsignallength = widgets.Text(value='0', disabled=True)

selectsecondsignal = widgets.Select(
    options=files,
    description='Select Second File:',
    disabled=False
)

secondsignallength = widgets.Text(value="0", disabled=True)

selectfirstsignal.observe(onchangeselectfirstsignal)
selectsecondsignal.observe(onchangeselectsecondsignal)

display(selectfirstsignal, firstsignallength, selectsecondsignal, secondsignallength)